# Setup

In [0]:
# install transformers package from Hugging Face transformers package contains 
# pre-trained BERT model and other useful interfaces
!pip install transformers 

     |████████████████████████████████| 542kB 25.2MB/s 
     |████████████████████████████████| 1.0MB 65.5MB/s 
     |████████████████████████████████| 3.7MB 61.3MB/s 
     |████████████████████████████████| 870kB 21.0MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=877785d7f788d7c104497fc916ee5dcec6a38d194aa0f9fd972757160ef42717
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
# Check if Colab's GPU is available and set up the GPU device
import tensorflow as tf
import torch

device_name = tf.test.gpu_device_name()

if device_name == '/device:GPU:0':
    print('GPU is available')
else:
    raise SystemError('No GPU device available')

if torch.cuda.is_available():
    device = torch.device("cuda")
    print('GPU name and type:', torch.cuda.get_device_name(0))
else:
    print('No GPU available, using CPU')
    device = torch.device("cpu")

GPU is available
GPU name and type: Tesla P4


# Data Preprocessing

In [0]:
# dataset pre-processing
# load data_extraction.ipynb
#### TO-DO fill in tweets & labels

tweets = None
labels = None

# Training Prep

### Input Tokenization & Encoding

In [0]:
from transformers import BertTokenizer

print('Loading BERT tokenizer...')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

# visualize BERT tokenization by an example
print('Before:', tweets[0])
print('After(words):', tokenizer.tokenize(tweets[0]))
print('After(ids):', tokenizer.convert_tokens_to_ids(tokenizer.tokenize(tweets[0])))

Loading BERT tokenizer...


In [0]:
# Get the maximum length of the dataset since all input instances have to have a constant length
max_len = 0

for tweet in tweets:
    max_len = max(max_len, len(tweet))

max_len += 2
print('Max tweet length:', max_len)


# tokenize all tweets, acquire corresponding token ids and attention masks
input_ids = []
attention_masks = []

for tweet in tweets:
    encoded_dict = tokenizer.encode_plus(tweet, 
                                         add_special_tokens=True, 
                                         max_length=max_len,
                                         pad_to_max_length=True,
                                         return_attention_mask=True,
                                         return_tensors='pt'
                                         )
    

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
labels = torch.tensor(labels)

# visualize tweet, tokenized ids and attention masks
print('Original tweet: ', tweets[0])
print('Token IDs:', input_ids[0])
print('Attention Mask:', attention_masks[0])


### Training & Validation Split

In [0]:
from torch.utils.data import TensorDataset, random_split, DataLoader, RandomSampler, SequentialSampler

dataset = TensorDataset(input_ids, attention_masks, labels)

# training 80% validation 10% testing 10%
train_size = int(0.8 * len(dataset))
val_size = int(0.1 * len(dataset))
test_size = len(dataset) - train_size - val_size

print('training dataset size:', train_size)
print('validation dataset size:', val_size)
print('testing dataset size:', test_size)

train_ds, val_ds, test_ds = random_split(dataset, [train_size, val_size, test_size])

# create a dataloader
batch_size = 32

train_dl = DataLoader(train_ds, 
                      sampler=RandomSampler(train_ds), 
                      batch_size=batch_size)

val_dl = DataLoader(val_ds, 
                    sampler=SequentialSampler(val_ds), 
                    batch_size=batch_size)

test_dl = DataLoader(test_ds, 
                    sampler=SequentialSampler(test_ds), 
                    batch_size=batch_size)


### Model Visualization

In [0]:
from transformers import BertForSequenceClassification, BertConfig

model = BertForSequenceClassification.from_pretrained("bert-base-uncased", 
                                                      num_labels=11, 
                                                      output_attentions=False, 
                                                      output_hidden_states=False)

model.cuda()

#visualize
params = list(model.named_parameters())

for p in params:
    print("{:<60} {:>15}".format(p[0], str(tuple(p[1].size()))))


bert.embeddings.word_embeddings.weight                          (30522, 768)
bert.embeddings.position_embeddings.weight                        (512, 768)
bert.embeddings.token_type_embeddings.weight                        (2, 768)
bert.embeddings.LayerNorm.weight                                      (768,)
bert.embeddings.LayerNorm.bias                                        (768,)
bert.encoder.layer.0.attention.self.query.weight                  (768, 768)
bert.encoder.layer.0.attention.self.query.bias                        (768,)
bert.encoder.layer.0.attention.self.key.weight                    (768, 768)
bert.encoder.layer.0.attention.self.key.bias                          (768,)
bert.encoder.layer.0.attention.self.value.weight                  (768, 768)
bert.encoder.layer.0.attention.self.value.bias                        (768,)
bert.encoder.layer.0.attention.output.dense.weight                (768, 768)
bert.encoder.layer.0.attention.output.dense.bias                      (768,)

### Optimizer & Learning Rate Scheduler

In [0]:
from transformers import get_linear_schedule_with_warmup, AdamW

# set up the optimizer
optimizer = AdamW(model.parameters(),
                  lr=2e-5,
                  eps=1e-8)

# set up the lr scheduler
epochs = 3
total_steps = len(train_dl) * epochs

scheduler = get_linear_schedule_with_warmup(optimizer,
                                            num_warmup_steps=0,
                                            num_training_steps=total_steps)

### Helper Functions

In [0]:
import numpy as np
import time
import datetime

# calculate accuracy
# TO-DO
def get_accuracy(preds, labels):
    return None

# take a time in seconds and returns a string hh:mm:ss
def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    
    return str(datetime.timedelta(seconds=elapsed_rounded))


# Training

In [0]:
import random

seed_val = 1
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

training_stats = []

total_t0 = time.time() 

for epoch_i in range(0, epochs):
    print("")
    print('======= Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
    print("")

    t0 = time.time()
    total_train_loss = 0
    model.train()

    for step, batch in enumerate(train_dl):
        # periodically update elapsed time
        ## TO-DO, change progess update param
        if step % 50 == 0 and not step == 0:
            elapsed = format_time(time.time() - t0)

            print('Batch {:>5,} of {:>5,}. Elapsed: {:}.'.format(step, 
                                                                 len(train_dl), 
                                                                 elapsed))
        # unpack current batch's input & labels   
        cur_input_ids = batch[0].to(device)
        cur_input_mask = batch[1].to(device)
        cur_labels = batch[2].to(device)

        # clear previously calculated gradients
        model.zero_grad()

        # perform a forward pass
        # logits = classifications, before activation function e.x. softmax
        loss, logits = model(cur_input_ids, 
                             token_type_ids=None,
                             attention_mask=cur_input_mask,
                             labels=cur_labels)
        
        # accumulate training loss
        total_train_loss += loss.item()

        # perform a backward pass to calculate the gradients of params
        loss.backward()

        # clip the gradients if it is not in [-1,1]
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        # update weights
        optimizer.step()

        # update lr
        scheduler.step()

    # calculate stats after 1 epoch of training
    avg_train_loss = total_train_loss / len(train_dl)
    training_time = format_time(time.time() - t0)

    print("")
    print("  Average training loss: {0:.2f}".format(avg_train_loss))
    print("  Training epcoh took: {:}".format(training_time))

    # validation 
    print("")
    print("Validating...")

    t0 = time.time()

    model.eval()

    total_eval_accuracy = 0
    total_eval_loss = 0
    nb_eval_steps = 0

    for batch in validation_dl:
        
        # unpack current batch's input and labels
        cur_input_ids = batch[0].to(device)
        cur_input_mask = batch[1].to(device)
        cur_labels = batch[2].to(device)

        # no need to calculate and trace gradient
        with torch.no_grad():
            (loss, logits) = model(cur_input_ids,
                                   token_type_ids=None,
                                   attention_mask=cur_input_mask,
                                   labels=cur_labels)
                    
        # no need to calculate prediction & labels in gpu
        # good practice when involves with large-scale dataset
        logits = logits.detach().cpu().numpy()
        label_ids = cur_labels.to('cpu')

        # calculate this batch's accuracy & loss and accumulate with other 
        # batches' accuracies & loss
        total_eval_accuracy += get_accuracy(logits, label_ids)
        total_eval_loss += loss.item()

    # average out loss and accuracy across all batches
    avg_val_accuracy = total_eval_accuracy / len(validation_dl)
    avg_val_loss = total_eval_loss / len(validation_dl)
    validation_time = format_time(time.time() - t0)

    print(
        "Average Accuracy: {0:.2f}, Loss: {1:.2f}, Time elapsed: {2:.2f}".format(
            avg_val_accuracy, avg_val_loss, validation_time
        )
    )

    # record all statistics for this epoch
    training_stats.append({
        'epoch': epoch_i + 1,
        'Training Loss': avg_train_loss,
        'Valid. Loss': avg_val_loss,
        'Valid. Accur': avg_val_accuracy,
        'Training Time': training_time,
        'Validation Time': validation_time
    })


print("")
print("Training Completed. Training took {:}".format(format_time(time.time() - total_t0)))


In [0]:
# Display training stats
from pandas as pd

# Display numbers with two decimal
pd.set_option('precision', 2)

df_stats = pd.DataFrame(data=training_stats)
df_stats = df_stats.set_index('epoch')

df_stats

In [0]:
# Visualize training stats
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

sns.set(style='darkgrid')
sns.set(font_scale=1.5)
plt.rcParams["figure.figsize"] = (12, 6)

plt.plot(df_stats['Training Loss'], 'b-o', label="Training")
plt.plot(df_stats['Validation Loss'], 'g-o', label="Validation")

plt.title("Training & Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()

### TO_DO: change epoch number
plt.xticks([1,2,3,4])

plt.show()

# Testing

In [0]:
print('Predicting labels for {:,} test sentences...'.format(len(test_dl)))

model.eval()

total_accuracy_test = 0

for batch in test_dl:
    # add batch to GPU
    batch = tuple(t.to(device) for t in batch)

    cur_input_ids, cur_input_mask, cur_labels = batch

    with torch.no_grad():
        outputs = model(cur_input_ids, 
                        token_type_ids=None, 
                        attention_mask=cur_input_mask)
    logits = outputs[0]

    # Move logits and labels to CPU
    logits = logits.detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    total_accuracy_test += get_accuracy(logits, label_ids)

print('Testing Accuracy: {:.2f}'.format(total_accuracy_test/len(test_dl)))
print('Testing Completed')


# Conclusion